# Gradient Boosting classifier with decision trees

Gradient boosting is an ensemble learning technique that consists of sequentially adding predictors. Each predictor will fit on the residual error of the previous predictor.

The predictors used in this case are weak learners decision trees.

In [1]:
library(xgboost)
library(caret)

Loading required package: lattice

Loading required package: ggplot2



In [2]:
column_to_drop<-c("X.1","X")
training_set <- read.csv("../Data/PreProcess/processed_training_data_split.csv") # load training set
validation_set <- read.csv("../Data/PreProcess/processed_verification_data_split.csv") # load validation set

# Dropped "X.1","X" because they just represent the row numbers
training_set<-training_set[,!(names(validation_set) %in% column_to_drop)] # drop the desired columns on validation set
validation_set<-validation_set[,!(names(validation_set) %in% column_to_drop)] # drop the desired columns on validation set

In [ ]:
## test GBM - fait planter le kernel 
library(gbm)
gbm.fit <- gbm(
  formula = id ~ .,
  distribution = "gaussian",
  data = training_set,
  n.trees = 10,
  interaction.depth = 1,
  shrinkage = 0.9,
  cv.folds = 5,
  verbose = FALSE
  )  

Using  trees...




In [3]:
## test xgboost
modelGB <- train(
  id ~., data = training_set, method = "xgbTree",
  trControl = trainControl("cv", number = 10)
  )
# tuning parameter
modelGB$bestTune

,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102,150,3,0.4,0,0.8,1,0.5


In [4]:
modelGB
save(modelGB,file = "4-Models/modelGB.RData")

eXtreme Gradient Boosting 

47289 samples
   83 predictor
    3 classes: 'functional', 'functional needs repair', 'non functional' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 42560, 42561, 42559, 42561, 42560, 42561, ... 
Resampling results across tuning parameters:

  eta  max_depth  colsample_bytree  subsample  nrounds  Accuracy   Kappa    
  0.3  1          0.6               0.50        50      0.7126180  0.4180032
  0.3  1          0.6               0.50       100      0.7211823  0.4408679
  0.3  1          0.6               0.50       150      0.7249252  0.4513928
  0.3  1          0.6               0.75        50      0.7119624  0.4165799
  0.3  1          0.6               0.75       100      0.7204421  0.4386828
  0.3  1          0.6               0.75       150      0.7241851  0.4491404
  0.3  1          0.6               1.00        50      0.7115183  0.4147708
  0.3  1          0.6               1.00       100      0.7196174  0.4363322

In [5]:
load("4-Models/modelGB.RData")
pred <- predict(modelGB,validation_set)
conf <- confusionMatrix(data=pred, reference = factor(validation_set$id[-1])) # GBM confusion matrix
conf

Confusion Matrix and Statistics

                         Reference
Prediction                functional functional needs repair non functional
  functional                    5438                     642           1963
  functional needs repair         55                      80             34
  non functional                1030                     195           2673

Overall Statistics
                                         
               Accuracy : 0.6764         
                 95% CI : (0.668, 0.6847)
    No Information Rate : 0.5386         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.3741         
                                         
 Mcnemar's Test P-Value : < 2.2e-16      

Statistics by Class:

                     Class: functional Class: functional needs repair
Sensitivity                     0.8337                       0.087241
Specificity                     0.5337                       0.9920